In [1]:
import pickle
from gensim import corpora, models
import numpy as np

In [2]:
img_by_topic = np.load('../topic_models_crop/data2/50-topic-res-all-1m-np.npy')
print 1

1


In [3]:
# get full list of images
img_list = np.load('../topic_models_crop/data/image_list_yfcc100m_output_1m_crop.npy')
print 2

2


In [4]:
# get top 5000 images in topic animals
topic_animals_img_indices = [int(img_by_topic[35][i][0]) for i in range(0, 10000)]
# get urls of topic animal images
animal_image_urls = img_list[topic_animals_img_indices]

np.save('data/img_list_animals',animal_image_urls)

print 3

3


In [5]:
# get binary yfcc100 vector 
yfcc_binary = np.load('../topic_models_crop/data/one_hot_yfcc100m_1m.npy')
print 4

4


In [6]:
# get binary vectors corresponding to topic animals
animals_binary = yfcc_binary[topic_animals_img_indices]

np.save('data/animals_binary',animals_binary)

print 5

5


In [7]:
X = np.load('data/animals_binary.npy')

print 6

6


In [19]:
num_topics = [2,50]
num_words = 4096 # Number of top features to be displayed per topic
num_images = X.shape[0]
print 'shape of data is:',X.shape

print 6

shape of data is: (10000, 4096)
6


In [9]:
# get corpus corresponding to images in topic animals
corpus = [[(j, X[i, j]) for j in range(num_words) if X[i, j]==1] for i in range(num_images)]

print 7

7


In [10]:
corpora.MmCorpus.serialize('data/corpus_animals.mm', corpus)

print 8

8


In [11]:
# Load corpus
corpus = corpora.MmCorpus('data/corpus_animals.mm')

print 9

9


In [12]:
animal_image_urls = np.load('data/img_list_animals.npy')

print 10

10


In [20]:
for K in num_topics:
    print K
    # Create the Topic Model
    model_name = str(K) + '-topics.model'
    lda = models.ldamodel.LdaModel(corpus, num_topics = K,  minimum_probability=0)
    lda.save('results/' + model_name)

    # Get topic for each image
    img_by_topic = [[] for _ in range(K)]
    for i in range(num_images):
        sorted_topics = sorted(lda.get_document_topics(corpus[i]), key=lambda x:x[1])
        for item in sorted_topics:
            img_by_topic[item[0]].append((i,item[1]))


    for j in range(K):
        img_by_topic[j].sort(key=lambda x: -x[1])

    with open("results/" + str(K) + "-topic-res.txt", "wb") as fp:
        pickle.dump(img_by_topic, fp)

2
50
